In [247]:
import pandas as pd 
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [248]:
#ccba
ccba = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_ccba_full_hashed.csv')
#cdtx
cdtx = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_cdtx0001_full_hashed.csv')
#custinfo
cdtx = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_custinfo_full_hashed.csv')
#dp
dp = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_dp_full_hashed.csv')
#remit
remit = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_train_x_remit1_full_hashed.csv')
#alert_time
alert_time = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/public_x_alert_date.csv')
#y
y = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/train_y_answer.csv')
#案件名單
doc = pd.read_csv('/content/drive/MyDrive/訓練資料集_first/預測的案件名單及提交檔案範例.csv')

In [ ]:
alert_dp = dp[dp['cust_id'].isin(cdtx[cdtx['alert_key'].isin(alert_time['alert_key'].tolist())]['cust_id'].tolist())]
train_dp = dp[~dp['cust_id'].isin(cdtx[cdtx['alert_key'].isin(alert_time['alert_key'].tolist())]['cust_id'].tolist())]
train_dp['y'] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
sar_idx = train_dp[train_dp['cust_id'].isin(cdtx[cdtx['alert_key'].isin(y[y['sar_flag']==1]['alert_key'].tolist())]['cust_id'].tolist())].index
train_dp.loc[sar_idx, 'y'] = 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
#Data Cleansing
#SAR戶僅留前X天
def sar_pred_d(x, day):
  prev_d = x.groupby('cust_id')['tx_date'].max() - day
  prev_d = prev_d.reset_index()
  prev_d.rename(columns={'tx_date':'prev_d'}, inplace=True)
  x = x.merge(prev_d, on='cust_id', how='left')
  x.drop(x[x['tx_date']<x['prev_d']].index,inplace=True)
  x.pop('prev_d')
  return x

train_dp = sar_pred_d(train_dp, 30)
alert_dp = sar_pred_d(alert_dp, 30)


In [ ]:
#特徵前處理
def preprocess(data):
  dict1 = {}
  idx = 0
  num = 0
  for i in range(0,395,1):
    dict1[i] = str(idx)
    num += 1
    if num == 7:
      idx += 1
      num = 0
  data['tx_date_group'] = data.tx_date.map(lambda x: dict1[x])
  data['session_cust_id'] = data.tx_date_group + data.cust_id
  data['date_cust_id'] = data.tx_date.astype(str) + data.cust_id
  data['date_time_cust_id'] = data.tx_date.astype(str) + data.tx_date.astype(str) + data.cust_id
  return data

train_dp = preprocess(train_dp)
alert_dp = preprocess(alert_dp)


In [ ]:
#session 交易差額比率
def amt_diff_func(data):
  session_amt_diff = data.groupby(['session_cust_id','debit_credit'])['tx_amt'].sum().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='session_cust_id', columns='debit_credit', values='tx_amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['session_amt_diff_ratio'] = \
    abs(session_amt_diff['CR'] - session_amt_diff['DB']) / abs(session_amt_diff['CR'] + session_amt_diff['DB'])
  session_amt_diff = session_amt_diff.reset_index()[['session_cust_id','session_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='session_cust_id', how='left')
#當日 交易差額比率
  date_amt_diff = data.groupby(['date_cust_id','debit_credit'])['tx_amt'].sum().reset_index()
  date_amt_diff = pd.pivot_table(date_amt_diff, index='date_cust_id', columns='debit_credit', values='tx_amt')
  date_amt_diff.fillna(1, inplace=True)
  date_amt_diff['date_amt_diff_ratio'] = \
  abs(date_amt_diff['CR'] - date_amt_diff['DB']) / abs(date_amt_diff['CR'] + date_amt_diff['DB'])
  date_amt_diff = date_amt_diff.reset_index()[['date_cust_id','date_amt_diff_ratio']]
  data = data.merge(date_amt_diff, on=['date_cust_id'], how='left')
#當時 交易差額比率
  date_time_amt_diff = data.groupby(['date_time_cust_id','debit_credit'])['tx_amt'].sum().reset_index()
  date_time_amt_diff = pd.pivot_table(date_time_amt_diff, index='date_time_cust_id', columns='debit_credit', values='tx_amt')
  date_time_amt_diff.fillna(1, inplace=True)
  date_time_amt_diff['date_time_amt_diff_ratio'] = \
  abs(date_time_amt_diff['CR'] - date_time_amt_diff['DB']) / abs(date_time_amt_diff['CR'] + date_time_amt_diff['DB'])
  date_time_amt_diff = date_time_amt_diff.reset_index()[['date_time_cust_id','date_time_amt_diff_ratio']]
  data = data.merge(date_time_amt_diff, on=['date_time_cust_id'], how='left')
  return data

train_dp = amt_diff_func(train_dp)
alert_dp = amt_diff_func(alert_dp)

In [ ]:
def tx_cnt_branch_func(data):
#當時交易筆數 tx_cnt_date_time
  tx_cnt_date_time = data.groupby(['cust_id','tx_date','tx_time'])['debit_credit'].count().reset_index()
  tx_cnt_date_time.rename(columns={'debit_credit':'tx_cnt_date_time'}, inplace=True)
  data = data.merge(tx_cnt_date_time, on=['cust_id','tx_date','tx_time'], how='left')
#當日交易筆數 tx_cnt_date
  tx_cnt_date = data.groupby(['cust_id','tx_date'])['debit_credit'].count().reset_index()
  tx_cnt_date.rename(columns={'debit_credit':'tx_cnt_date'}, inplace=True)
  data = data.merge(tx_cnt_date, on=['cust_id','tx_date'], how='left')
#當時總分行數 txbranch_day_cnt
  txbranch_day_time_cnt = data.groupby(['cust_id','tx_date','tx_time'])['txbranch'].count().reset_index()
  txbranch_day_time_cnt.rename(columns={'txbranch':'txbranch_day_time_cnt'}, inplace=True)
  data = data.merge(txbranch_day_time_cnt, on=['cust_id','tx_date','tx_time'], how='left')
#單日總分行數 txbranch_day_cnt
  txbranch_day_cnt = data.groupby(['cust_id','tx_date'])['txbranch'].count().reset_index()
  txbranch_day_cnt.rename(columns={'txbranch':'txbranch_day_cnt'}, inplace=True)
  data = data.merge(txbranch_day_cnt, on=['cust_id','tx_date'], how='left')
  return data
train_dp = tx_cnt_branch_func(train_dp)
alert_dp = tx_cnt_branch_func(alert_dp)

In [ ]:
def atm_txn_func(data):
#當日ATM 佔交易數比例
  day_atm_txn_ratio = data.groupby(['cust_id','tx_date'])['ATM'].sum().reset_index()
  day_atm_txn_ratio.rename(columns={'ATM':'day_atm_txn_ratio'}, inplace=True)
  data = data.merge(day_atm_txn_ratio, on=['cust_id','tx_date'], how='left')
  data.day_atm_txn_ratio = data.day_atm_txn_ratio / data.tx_cnt_date
#當時ATM 佔交易數比例
  day_time_atm_txn_ratio = data.groupby(['cust_id','tx_date','tx_time'])['ATM'].sum().reset_index()
  day_time_atm_txn_ratio.rename(columns={'ATM':'day_time_atm_txn_ratio'}, inplace=True)
  data = data.merge(day_time_atm_txn_ratio, on=['cust_id','tx_date','tx_time'], how='left')
  data.day_time_atm_txn_ratio = data.day_time_atm_txn_ratio / data.tx_cnt_date_time
  return data
train_dp = atm_txn_func(train_dp)
alert_dp = atm_txn_func(alert_dp)

In [ ]:
train_col = ['cust_id','session_amt_diff_ratio', 'date_amt_diff_ratio',
       'date_time_amt_diff_ratio', 'tx_cnt_date_time', 'tx_cnt_date',
       'txbranch_day_time_cnt', 'txbranch_day_cnt', 'day_atm_txn_ratio',
       'day_time_atm_txn_ratio','y']
x_col = train_col[1:-1]
train_data = train_dp[train_col].drop_duplicates()

In [ ]:
Y = train_data['y']
X = train_data[x_col]

In [ ]:
#model training 1
#逐筆交易處理
test_size = 0.2
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
#previous 30days label of 1
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))

Accuracy: 94.02%
1.0
0.14396887159533073


In [ ]:
#彙整model training 1偵測結果
#predict_proba以0.05機率區間為一個欄位判斷分佈
def mapping(proba):
    if proba <= 0.05:
        return 1
    elif 0.05 < proba <= 0.1:
        return 2
    elif 0.1 < proba <= 0.15:
        return 3
    elif 0.15 < proba <= 0.2:
        return 4
    elif 0.2 < proba <= 0.25:
        return 5
    elif 0.25 < proba <= 0.3:
        return 6
    elif 0.3 < proba <= 0.35:
        return 7
    elif 0.35 < proba <= 0.4:
        return 8
    elif 0.4 < proba <= 0.45:
        return 9
    elif 0.45 < proba <= 0.5:
        return 10
    elif 0.5 < proba <= 0.55:
        return 11
    elif 0.55 < proba <= 0.6:
        return 12
    elif 0.6 < proba <= 0.65:
        return 13
    elif 0.65 < proba <= 0.7:
        return 14
    elif 0.7 < proba <= 0.75:
        return 15
    elif 0.75 < proba <= 0.8:
        return 16
    elif 0.8 < proba <= 0.85:
        return 17
    elif 0.85 < proba <= 0.9:
        return 18
    elif 0.9 < proba <= 0.95:
        return 19
    elif 0.95 < proba <= 1:
        return 20

In [ ]:
train_data['proba'] = model.predict_proba(X)[:,1]

In [ ]:
result = train_data[['cust_id','proba','y']]
result['level'] = result["proba"].map(mapping)
result = result[['cust_id', 'level', 'y']]
result = result.groupby(['cust_id','level']).count().reset_index().pivot_table(index='cust_id', columns='level', values='y')
result.fillna(0, inplace=True)
result = result.div(result.sum(axis=1), axis=0)
result = result.reset_index()
for col in list(set([num for num in range(1,21,1)]) -  set(result.columns[1:])):
  result[col] = 0.0
result = result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
result = result.merge(train_data[['cust_id','y']].drop_duplicates(), on='cust_id', how='left')
result.pop('cust_id')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


level,cust_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,20
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,0.666667,0.333333,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,002f13f534d0bd9fcc1848db3ff5c7b1179d117f8de805...,0.333333,0.666667,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0031e83ddef78e57d17ac5dce088627b7745be8ff6cffc...,0.000000,0.666667,0.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0032adaf4a8731b4d6a3fdcba32f9ebdcd336490eda155...,0.000000,0.227273,0.681818,0.090909,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5485,ffae7fef923c78298cd647a3d573e48cc1404b07ac794a...,0.166667,0.833333,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5486,ffdbc1a5135cb6feddf7355e5656abf6973f21639c7b9b...,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5487,ffdd3f98b841d4d0ef58ad0d63abddac83ffb5babf973c...,0.666667,0.000000,0.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5488,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,0.500000,0.500000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#model training 2
#歸戶判斷是否報SAR
result_col = list(result.columns)
result_col.remove('y')
test_size = 0.2
seed = 42
X_train, X_test, y_train, y_test = train_test_split(result[result_col], result['y'], test_size=test_size, random_state=seed)
model_2nd = XGBClassifier()
model_2nd.fit(X_train, y_train)
y_pred = model_2nd.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))

Accuracy: 96.27%
0.5
0.12195121951219512


In [ ]:
#Submission
alert_x = alert_dp[x_col]

In [ ]:
alert_data = alert_dp
alert_data['proba'] = model.predict_proba(alert_x)[:,1]
alert_result = alert_data[['cust_id','proba']]
alert_result['level'] = alert_result["proba"].map(mapping)
alert_result = alert_result[['cust_id', 'level']]
alert_result['cnt'] = 1
alert_result = alert_result.groupby(['cust_id','level'])['cnt'].count().reset_index().pivot_table(index='cust_id', columns='level', values='cnt')
alert_result.fillna(0, inplace=True)
alert_result = alert_result.div(alert_result.sum(axis=1), axis=0)
alert_result = alert_result.reset_index()
for col in list(set([num for num in range(1,21,1)]) -  set(alert_result.columns[1:])):
  alert_result[col] = 0.0
alert_result = alert_result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
#alert_result.pop('cust_id')
alert_pred = model_2nd.predict_proba(alert_result[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]])
# evaluate predictions
final = alert_result[['cust_id']]
final['probability'] = alert_pred[:,1]
final

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


level,cust_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0172056578071e83399216fcd640bdc4de4583149d0fd9...,0.869565,0.130435,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,01b1735c3dbf0a35c1c989bb45434ce4326b365b73db06...,0.133333,0.600000,0.000000,0.266667,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,01c705b457297805c1c08137cb3adb164b8f393c265db2...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.059589,0.940411
4,022bec0a12b56285bf12c49ba1512c0acdbf4176a30d96...,0.875000,0.125000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,0.000000,0.189944,0.162011,0.491620,0.156425,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
702,ff538fed253fce00e25a7c1557ee7728a5d8dc9c1bbfc1...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
703,ff824c1a876b074a3535308f9a95649ef26c14b91433b8...,0.937500,0.062500,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
704,ffe5f2bd86ecfd7d0a0f4c4b76dd5c312649be95eb8b42...,0.086207,0.086207,0.258621,0.362069,0.103448,0.103448,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [ ]:
final = alert_time.merge(cdtx[['alert_key', 'cust_id']].merge(final, on='cust_id'), on='alert_key', how='left')[['alert_key', 'probability']]

In [ ]:
final.fillna(0, inplace=True)
final

,alert_key,probability
0,352342,0.124638
1,352866,0.023943
2,352696,0.152648
3,352330,0.048633
4,352683,0.284548
...,...,...
1840,364472,0.065475
1841,364788,0.034448
1842,364673,0.297682
1843,364626,0.644214


In [245]:
final.to_csv('/content/drive/MyDrive/訓練資料集_first/submission.csv')

In [227]:
alert_time

,alert_key,date
0,352342,365
1,352866,365
2,352696,365
3,352330,365
4,352683,365
...,...,...
1840,364472,393
1841,364788,393
1842,364673,393
1843,364626,393


In [229]:
doc['alert_key'].value_counts()

357307    1
371610    1
360786    1
367364    1
367546    1
         ..
376478    1
369975    1
360846    1
376908    1
372119    1
Name: alert_key, Length: 3850, dtype: int64

In [242]:
doc = doc[['alert_key']]
doc

,alert_key
0,357307
1,376329
2,373644
3,357668
4,354443
...,...
3845,364485
3846,363155
3847,368710
3848,358067


In [244]:
final = doc.merge(final, on='alert_key', how='left')
final.fillna(0,inplace=True)
final

,alert_key,probability
0,357307,0.030644
1,376329,0.000000
2,373644,0.000000
3,357668,0.024843
4,354443,0.152648
...,...,...
3845,364485,0.023200
3846,363155,0.266375
3847,368710,0.000000
3848,358067,0.392237


In [239]:
doc = doc.merge(alert_time, on='alert_key', how='left')
doc.pop('probability')
doc


alert_key             0
date               2005
cust_id            2005
risk_rank          2005
occupation_code    2005
total_asset        2005
AGE                2005
dtype: int64